## Scraper

Import modules

In [2]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

import datetime
import sched, time

In [18]:
datetime.datetime.now().hour

1

Define the url to be scraped and load it

In [3]:
url='https://www.bilibili.com/'
html = urlopen(url).read().decode('utf-8')

Define a list of all sections on front page

In [4]:
secs = {'home-top':'groom-module home-card', 'home-pop':'home_popularize', 'live':'card-live-module'}

Load source to Soup

In [5]:
soup = BeautifulSoup(html, features='lxml')

In [6]:
# print(soup)

In [7]:
groom = soup.find_all('div',{'class': 'groom-module home-card'})

In [13]:
[i.a['href'].split('av')[1][:-1] for i in groom]

['22734487',
 '22766666',
 '22688326',
 '22736459',
 '22684448',
 '22739609',
 '22731605',
 '22743431']

## Write functions

In [4]:
baseurl='https://www.bilibili.com/'

def groomjob(url):
    html = urlopen(url).read().decode('utf-8')

    soup = BeautifulSoup(html, features='lxml')
    groom = soup.find_all('div',{'class': 'groom-module home-card'})

    aid = [i.a['href'].split('av')[1][:-1] for i in groom]
    return aid
    

def recordgroom():
    dt = str(datetime.datetime.now() )
    with open("data.txt", "a") as f:
        f.write( dt + ',' )
        [f.write(i+',') for i in aid]
        f.write('\n')


hourflag = datetime.datetime.now().hour

# while True:
    
#     if datetime.datetime.now().hour > hourflag:
    
#         print('Job Running')

#         recordgroom()

#         hourflag = datetime.datetime.now().hour
    
            


## Test scheduler

In [1]:
import sched, time
s = sched.scheduler(time.time, time.sleep)

In [2]:
def print_time(a='default'):
    cur_time = time.time()
    cur_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(cur_time))
    print("From print_time", cur_time, a)

def print_some_times():
    print_time()
    s.enter(10, 1, print_time)
    s.enter(5, 2, print_time, argument=('positional',))
    s.enter(5, 1, print_time, kwargs={'a': 'keyword'})
    s.run()
    print(time.time())

print_some_times()



From print_time 2018-05-02 19:10:38 default
From print_time 2018-05-02 19:10:43 positional
From print_time 2018-05-02 19:10:43 keyword
From print_time 2018-05-02 19:10:48 default
1525309848.8464918


In [3]:
s.empty()

True

In [13]:
dict = {}

In [78]:
def cronsched(scheduler, interval, dic, iden, action, actionargs=()):

    action(*actionargs)
    ev = scheduler.enter(interval, 1, cronsched,
                    (scheduler, interval, dic, iden, action, actionargs))
    
    dic[iden] = ev
    
def croncancel( scheduler, interval, eve ):
    
    scheduler.enter( interval, 1, scheduler.cancel, eve )

In [79]:
cronsched( s, 1, dict, 'new_vid', print_time )

From print_time 2018-05-02 19:30:05 default


In [80]:
dict

{'new_vid': Event(time=1525311006.819536, priority=1, action=<function cronsched at 0x112f34048>, argument=(<sched.scheduler object at 0x112e6b390>, 1, {'new_vid': Event(time=1525311006.819536, priority=1, action=<function cronsched at 0x112f34048>, argument=(...), kwargs={})}, 'new_vid', <function print_time at 0x112e6f950>, ()), kwargs={})}

In [81]:
s.queue

[Event(time=1525311006.819536, priority=1, action=<function cronsched at 0x112f34048>, argument=(<sched.scheduler object at 0x112e6b390>, 1, {'new_vid': Event(time=1525311006.819536, priority=1, action=<function cronsched at 0x112f34048>, argument=(...), kwargs={})}, 'new_vid', <function print_time at 0x112e6f950>, ()), kwargs={})]

In [86]:
croncancel( s, 4, dict['new_vid'] )

In [87]:
s.empty()

False

In [92]:
s.queue

[]

In [93]:
s.run()

In [94]:
s.cancel(s.queue[0])

IndexError: list index out of range